# Classification de textes avec les Transformers

<p align="center">
  <a href="https://raw.githubusercontent.com/auduvignac/llm-finetuning/refs/heads/main/notebooks/project/finetuning-projet.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Ouvrir dans Google Colab"/>
  </a>
</p>

Ce projet porte sur une tâche de **classification de textes** appliquée au **jeu de données IMDB** (analyse de sentiments ou *sentimental analysis*).  
Ce travail s'appuiera sur les **architectures de type encodeur**, en particulier l’un des modèles les plus connus : **BERT** (et sa variante légère **DistilBERT**).

## Présentation

Le projet consiste à utiliser la librairie `datasets` pour le chargement des données et les `tokenizers` pour le prétraitement des textes.

La librairie **Transformers** propose une API simple pour utiliser des modèles pré-entraînés tels que **BERT** ou **GPT**. Elle facilite leur téléchargement, leur ré-entraînement et leur intégration, tout en réduisant les coûts de calcul et en restant compatible avec **PyTorch, TensorFlow et JAX**.

## Objectif du projet

Dans le cadre de ce projet, l’expérimentation portera sur la librairie **Hugging Face** afin de :
- Charger et adapter un modèle de type **BERT** à une tâche de classification de textes ;
- Évaluer ses performances sur le dataset **IMDB** ;
- Analyser les résultats et discuter des choix réalisés (modèle, preprocessing, paramètres, etc.).

Le travail sera guidé par les interrogations suivantes :

- Bien que tous ces modèles reposent sur l’architecture **Transformer**, quelles en sont les spécificités ?
- Quel format d’entrée est attendu par le modèle ?
- Quels types de sorties génère-t-il ?
- Le modèle peut-il être utilisé tel quel ou doit-il être adapté à la tâche considérée ?

Ces questions constituent une part essentielle du travail quotidien d’un chercheur en NLP et seront examinées dans le cadre de ce projet de *fine-tuning*.

In [1]:
!wget -q https://raw.githubusercontent.com/auduvignac/llm-finetuning/refs/heads/main/setup_env.py -O setup_env.py
%run setup_env.py

⚡ Exécution sur Colab : vérification des dépendances...
✅ datasets déjà présent.
✅ matplotlib déjà présent.
✅ numpy déjà présent.
✅ tabulate déjà présent.
✅ torch déjà présent.
✅ tqdm déjà présent.
✅ transformers déjà présent.


In [2]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
import matplotlib.pyplot as plt

from transformers import DistilBertTokenizer
from tqdm.notebook import tqdm


import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from torch.utils.data import DataLoader
from tabulate import tabulate
from datasets import load_dataset

from tqdm.notebook import tqdm

# If the machine you run this on has a GPU available with CUDA installed,
# use it. Using a GPU for learning often leads to huge speedups in training.
# See https://developer.nvidia.com/cuda-downloads for installing CUDA
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

## Chargement du jeu d'entraînement

La phase initiale consiste à procéder au chargement du jeu d'entraînement au moyen de l'instruction suivante :
```python
dataset = load_dataset("scikit-learn/imdb", split="train")
```

In [3]:
dataset = load_dataset("scikit-learn/imdb", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

IMDB Dataset.csv:   0%|          | 0.00/66.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Après le chargement, il convient de procéder à l'affichage du jeu de données afin d'en examiner la structure.

In [4]:
print(dataset)

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})


L'exécution de la commande `print(dataset)` renvoie la description suivante du jeu de données :

```
Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})
```

Le corpus est constitué de 50 000 instances, chacune décrite par deux attributs : le texte de la critique (*review*) et son étiquette de polarité (*sentiment*).
